In [1]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
  

# Configure OpenAI API

AZURE_OPENAI_API_VERSION = "2023-03-15-preview"
AZURE_OPENAI_ENDPOINT ="https://cog-42as6n6i4ldsi.openai.azure.com/"
AZURE_OPENAI_API_KEY ="653103af9cd04492a2232b70327b7ce0"

# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["OPENAI_API_TYPE"] = "azure"
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

In [2]:
import pandas as pd
import io
from azure.storage.blob import BlobServiceClient

# Define the connection string and container name
conn_str = "DefaultEndpointsProtocol=https;AccountName=entchatgptstr;AccountKey=gZss7VuVlbiwShlfML34GaeAw3QYEyicbZLF9D47zkGa/GCgx6jNGcNGBz0qqRa4Ci3MPqXrWI2O+AStm9gdMg==;EndpointSuffix=core.windows.net"
container_name = "pubmeddata"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get a reference to the blob container
container_client = blob_service_client.get_container_client(container_name)

# Get a reference to the blob
blob_client = container_client.get_blob_client("complaints.csv")

# Download the blob as a string
blob_string = blob_client.download_blob().content_as_text(encoding="utf-8")

# Load the string into a Pandas dataframe
df = pd.read_csv(io.StringIO(blob_string), encoding="utf-8")



/anaconda/envs/azureml_py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3221: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Filter out rows where Consumer complaint narrative is longer than 4000 words
df = df[df['Consumer complaint narrative'].str.len() <= 4000]

# Count the number of rows
num_rows = len(df)
# count na values in column "Consumer complaint narrative"
df['Consumer complaint narrative'].isna().sum()

#remove the rows where column "Consumer complaint narrative" is less than 10 characters
# count rows in df
print(len(df))
df = df[df['Consumer complaint narrative'].str.len() >= 30]
print(len(df))
df = df[df['Consumer complaint narrative'].notna()]
print(len(df))

1344868
1342697
1342697


In [4]:
# Create train and test sets
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, random_state=32, test_size=0.20, shuffle=True)

print(df_train.shape)
print(df_test.shape)

(1074157, 18)
(268540, 18)


In [5]:
# Task: Multi class classification
#Check if the classes are balaned in df_train
#df_test['Product'].value_counts()

# Take 40000 rows from df_train;
df_train = df_train.sample(n=40000, random_state=32)
print(df_train['Product'].value_counts())

# Take 10000 rows from df_test;
df_test = df_test.sample(n=10000, random_state=32)
df_test['Product'].value_counts()



Credit reporting, credit repair services, or other personal consumer reports    20554
Debt collection                                                                  6035
Mortgage                                                                         3025
Credit card or prepaid card                                                      2879
Checking or savings account                                                      2022
Student loan                                                                     1013
Credit reporting                                                                  942
Money transfer, virtual currency, or money service                                927
Vehicle loan or lease                                                             723
Credit card                                                                       567
Bank account or service                                                           441
Payday loan, title loan, or personal loan             

Credit reporting, credit repair services, or other personal consumer reports    5075
Debt collection                                                                 1574
Mortgage                                                                         744
Credit card or prepaid card                                                      721
Checking or savings account                                                      538
Student loan                                                                     268
Credit reporting                                                                 229
Money transfer, virtual currency, or money service                               221
Vehicle loan or lease                                                            172
Credit card                                                                      147
Payday loan, title loan, or personal loan                                        115
Bank account or service                                          

In [10]:
#check if df_train contains rows with null
df_train.isnull().sum()

Date received                       0
Product                             0
Sub-product                      1586
Issue                               0
Sub-issue                        6169
Consumer complaint narrative        0
Company public response         19595
Company                             0
State                             147
ZIP code                            0
Tags                            34471
Consumer consent provided?          0
Submitted via                       0
Date sent to company                0
Company response to consumer        0
Timely response?                    0
Consumer disputed?              35019
Complaint ID                        0
dtype: int64

In [6]:
#upload df_train and df_ test to azure blob storage
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
import uuid

# Define the connection string and container name
conn_str = "DefaultEndpointsProtocol=https;AccountName=entchatgptstr;AccountKey=gZss7VuVlbiwShlfML34GaeAw3QYEyicbZLF9D47zkGa/GCgx6jNGcNGBz0qqRa4Ci3MPqXrWI2O+AStm9gdMg==;EndpointSuffix=core.windows.net"
container_name = "pubmeddata"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get a reference to the blob container
container_client = blob_service_client.get_container_client(container_name)

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_train.csv")

# Upload the blob
blob_client.upload_blob(df_train.to_csv(index=False), overwrite=True)

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_test.csv")

# Upload the blob
blob_client.upload_blob(df_test.to_csv(index=False), overwrite=True)



{'etag': '"0x8DB81EC4551F754"',
 'last_modified': datetime.datetime(2023, 7, 11, 8, 53, 17, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b"\x9c\xad\'\xe4\xf57\xdbw\xb7\xb3\xc0\xc5\xdb\xb8\xd48"),
 'client_request_id': '606c8761-1fc8-11ee-8840-07237768152c',
 'request_id': 'e5d8b4c0-f01e-000e-7bd5-b365cb000000',
 'version': '2021-08-06',
 'version_id': None,
 'date': datetime.datetime(2023, 7, 11, 8, 53, 16, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

In [7]:
#download df_train and df_ test to azure blob storage
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os 
import pandas as pd

import io

# Define the connection string and container name
conn_str = "DefaultEndpointsProtocol=https;AccountName=entchatgptstr;AccountKey=gZss7VuVlbiwShlfML34GaeAw3QYEyicbZLF9D47zkGa/GCgx6jNGcNGBz0qqRa4Ci3MPqXrWI2O+AStm9gdMg==;EndpointSuffix=core.windows.net"
container_name = "pubmeddata"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get a reference to the blob container
container_client = blob_service_client.get_container_client(container_name)

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_train.csv")

# Download the blob as a string
blob_string = blob_client.download_blob().content_as_text(encoding="utf-8")

# Load the string into a Pandas dataframe
df_train = pd.read_csv(io.StringIO(blob_string), encoding="utf-8")

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_test.csv")

# Download the blob as a string
blob_string = blob_client.download_blob().content_as_text(encoding="utf-8")

# Load the string into a Pandas dataframe
df_test = pd.read_csv(io.StringIO(blob_string), encoding="utf-8")




In [8]:
#check if df_train contains rows with null value
df_train.isnull().sum()


Date received                       0
Product                             0
Sub-product                      1579
Issue                               0
Sub-issue                        6121
Consumer complaint narrative        0
Company public response         19444
Company                             0
State                             159
ZIP code                            0
Tags                            34466
Consumer consent provided?          0
Submitted via                       0
Date sent to company                0
Company response to consumer        0
Timely response?                    0
Consumer disputed?              34993
Complaint ID                        0
dtype: int64

In [50]:

import tiktoken

# Get the encoding
encoding = tiktoken.get_encoding('cl100k_base')

# Add a new column to the dataframe where you put the total token count of the row
df_c = df_test.assign(total_token_count=df_test['Consumer complaint narrative'].apply(lambda x: len(encoding.encode(str(x)))))

# Print the first 5 rows of the dataframe, then also the total number of tokens
total_tokens = df_c['total_token_count'].sum()

cost_for_embeddings = total_tokens / 1000 * 0.0004
print(f"Embedding the 'combined' column would cost ${cost_for_embeddings} for embeddings")


Embedding the 'combined' column would cost $0.8184404000000001 for embeddings


In [9]:
from langchain.embeddings import OpenAIEmbeddings
# Create a new column in df_train called Embeddings_complaint and use OpenAi embeddings to create the embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

#Create a new column in df_train called Embeddings_complaint and use embeddings.embed_query to create the embeddings; Embeddings are based on column "complaint" in df_train
df_train['Embeddings_complaint'] = df_train['Consumer complaint narrative'].apply(lambda x: embeddings.embed_query(str(x)))
df_train.head(10)



,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Embeddings_complaint
0,2023-01-25,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,My name is XXXX XXXX XXXX this complaint is no...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",VA,23234,NaN,Consent provided,Web,2023-01-25,Closed with non-monetary relief,Yes,NaN,6478164,"[-0.0073983436450362206, 0.006172811612486839,..."
1,2022-12-13,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,I reviewed my Consumer Reports and noticed tha...,NaN,"EQUIFAX, INC.",MN,557XX,NaN,Consent provided,Web,2022-12-13,Closed with explanation,Yes,NaN,6314072,"[-0.028294987976551056, -0.007571431342512369,..."
2,2020-09-17,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I am looking at my credit report and I have ac...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,361XX,NaN,Consent provided,Web,2020-09-17,Closed with explanation,Yes,NaN,3849473,"[-0.02628335729241371, -0.011320310644805431, ..."
3,2022-02-09,Credit card or prepaid card,General-purpose credit card or charge card,Closing your account,Company closed your account,I spoke with Citi via chat about my options fo...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",FL,322XX,NaN,Consent provided,Web,2022-02-09,Closed with non-monetary relief,Yes,NaN,5203831,"[-0.025717033073306084, -0.01054273173213005, ..."
4,2023-01-26,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"When I investigated my credit report, I realiz...",Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,35401,NaN,Consent provided,Web,2023-01-26,Closed with non-monetary relief,Yes,NaN,6486360,"[-0.01795455627143383, -0.006655335426330566, ..."
5,2015-03-26,Debt collection,Credit card,Communication tactics,Called after sent written cease of comm,Debt collector is consistently calling on priv...,NaN,PORTFOLIO RECOVERY ASSOCIATES INC,MD,21223,NaN,Consent provided,Web,2015-03-26,Closed with non-monetary relief,Yes,No,1303385,"[-0.034912336617708206, -0.02331012859940529, ..."
6,2021-10-26,Checking or savings account,Checking account,Managing an account,Problem accessing account,I have been locked out from online access and ...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",TX,76131,NaN,Consent provided,Web,2021-10-26,Closed with explanation,Yes,NaN,4847143,"[-0.020275797694921494, 0.0004513960739132017,..."
7,2018-03-01,Mortgage,Conventional home mortgage,Trouble during payment process,NaN,Rushmore violate The California Homeowner Bill...,NaN,RUSHMORE LOAN MANAGEMENT SERVICES LLC,CA,XXXXX,NaN,Consent provided,Web,2018-03-01,Closed with explanation,Yes,NaN,2829717,"[-0.03550923988223076, -0.0009103335905820131,..."
8,2023-01-09,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the Fair Credit Reporting a...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,19124,NaN,Consent provided,Web,2023-01-09,Closed with non-monetary relief,Yes,NaN,6416198,"[-0.022414909675717354, 0.0034802446607500315,..."
9,2019-09-17,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,XXXX formerly XXX

In [10]:
#Safe df_train to azure blob storage
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
import uuid

# Define the connection string and container name
conn_str = "DefaultEndpointsProtocol=https;AccountName=entchatgptstr;AccountKey=gZss7VuVlbiwShlfML34GaeAw3QYEyicbZLF9D47zkGa/GCgx6jNGcNGBz0qqRa4Ci3MPqXrWI2O+AStm9gdMg==;EndpointSuffix=core.windows.net"
container_name = "pubmeddata"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get a reference to the blob container
container_client = blob_service_client.get_container_client(container_name)

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_train_embeddings10k.csv")

# Upload the blob
blob_client.upload_blob(df_train.to_csv(index=False), overwrite=True)




{'etag': '"0x8DB81F7F9735926"',
 'last_modified': datetime.datetime(2023, 7, 11, 10, 17, 4, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'u{vl\xca\xfc\xa2n'),
 'client_request_id': '14665024-1fd4-11ee-8840-07237768152c',
 'request_id': 'abc77de6-901e-006a-44e0-b39453000000',
 'version': '2021-08-06',
 'version_id': None,
 'date': datetime.datetime(2023, 7, 11, 10, 17, 3, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

In [11]:
#Safe df_train to azure blob storage
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
import uuid

# Define the connection string and container name
conn_str = "DefaultEndpointsProtocol=https;AccountName=entchatgptstr;AccountKey=gZss7VuVlbiwShlfML34GaeAw3QYEyicbZLF9D47zkGa/GCgx6jNGcNGBz0qqRa4Ci3MPqXrWI2O+AStm9gdMg==;EndpointSuffix=core.windows.net"
container_name = "pubmeddata"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get a reference to the blob container
container_client = blob_service_client.get_container_client(container_name)

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_train_embeddings10k.csv")

#download the blob
blob_string = blob_client.download_blob().content_as_text(encoding="utf-8")

# Load the string into a Pandas dataframe
df_train = pd.read_csv(io.StringIO(blob_string), encoding="utf-8")

In [12]:
#calculate the null values in df_train per column
df_train.isnull().sum()

Date received                       0
Product                             0
Sub-product                      1579
Issue                               0
Sub-issue                        6121
Consumer complaint narrative        0
Company public response         19444
Company                             0
State                             159
ZIP code                            0
Tags                            34466
Consumer consent provided?          0
Submitted via                       0
Date sent to company                0
Company response to consumer        0
Timely response?                    0
Consumer disputed?              34993
Complaint ID                        0
Embeddings_complaint                0
dtype: int64

In [13]:
df_train.head(50)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Embeddings_complaint
0,2023-01-25,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,My name is XXXX XXXX XXXX this complaint is no...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",VA,23234,NaN,Consent provided,Web,2023-01-25,Closed with non-monetary relief,Yes,NaN,6478164,"[-0.0073983436450362206, 0.006172811612486839,..."
1,2022-12-13,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,I reviewed my Consumer Reports and noticed tha...,NaN,"EQUIFAX, INC.",MN,557XX,NaN,Consent provided,Web,2022-12-13,Closed with explanation,Yes,NaN,6314072,"[-0.028294987976551056, -0.007571431342512369,..."
2,2020-09-17,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I am looking at my credit report and I have ac...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,361XX,NaN,Consent provided,Web,2020-09-17,Closed with explanation,Yes,NaN,3849473,"[-0.02628335729241371, -0.011320310644805431, ..."
3,2022-02-09,Credit card or prepaid card,General-purpose credit card or charge card,Closing your account,Company closed your account,I spoke with Citi via chat about my options fo...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",FL,322XX,NaN,Consent provided,Web,2022-02-09,Closed with non-monetary relief,Yes,NaN,5203831,"[-0.025717033073306084, -0.01054273173213005, ..."
4,2023-01-26,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"When I investigated my credit report, I realiz...",Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,35401,NaN,Consent provided,Web,2023-01-26,Closed with non-monetary relief,Yes,NaN,6486360,"[-0.01795455627143383, -0.006655335426330566, ..."
5,2015-03-26,Debt collection,Credit card,Communication tactics,Called after sent written cease of comm,Debt collector is consistently calling on priv...,NaN,PORTFOLIO RECOVERY ASSOCIATES INC,MD,21223,NaN,Consent provided,Web,2015-03-26,Closed with non-monetary relief,Yes,No,1303385,"[-0.034912336617708206, -0.02331012859940529, ..."
6,2021-10-26,Checking or savings account,Checking account,Managing an account,Problem accessing account,I have been locked out from online access and ...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",TX,76131,NaN,Consent provided,Web,2021-10-26,Closed with explanation,Yes,NaN,4847143,"[-0.020275797694921494, 0.0004513960739132017,..."
7,2018-03-01,Mortgage,Conventional home mortgage,Trouble during payment process,NaN,Rushmore violate The California Homeowner Bill...,NaN,RUSHMORE LOAN MANAGEMENT SERVICES LLC,CA,XXXXX,NaN,Consent provided,Web,2018-03-01,Closed with explanation,Yes,NaN,2829717,"[-0.03550923988223076, -0.0009103335905820131,..."
8,2023-01-09,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the Fair Credit Reporting a...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,19124,NaN,Consent provided,Web,2023-01-09,Closed with non-monetary relief,Yes,NaN,6416198,"[-0.022414909675717354, 0.0034802446607500315,..."
9,2019-09-17,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,XXXX formerly XXX

In [55]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

#Create a new column in df_train called Embeddings_complaint and use embeddings.embed_query to create the embeddings; Embeddings are based on column "complaint" in df_train
df_test['Embeddings_complaint'] = df_test['Consumer complaint narrative'].apply(lambda x: embeddings.embed_query(str(x)))
df_test.head(10)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Embeddings_complaint
0,2021-09-17,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,MY NAME IS XXXX XXXX AND I'M SUBMITTING THIS C...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,93727,NaN,Consent provided,Web,2021-09-17,Closed with non-monetary relief,Yes,NaN,4729991,"[-0.0461684875190258, -0.018764231353998184, 0..."
1,2020-05-26,Debt collection,Credit card debt,Took or threatened to take negative or legal a...,Collected or attempted to collect exempt funds,COLLECTION AGENCIES DEMAND PAYMENT ON ACCOUNTS...,NaN,Resurgent Capital Services L.P.,CA,90018,NaN,Consent provided,Web,2020-05-26,Closed with explanation,Yes,NaN,3668941,"[-0.02608240209519863, -0.027123533189296722, ..."
2,2022-05-01,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the Fair Credit Reporting a...,NaN,"EQUIFAX, INC.",AZ,85041,NaN,Consent provided,Web,2022-05-01,Closed with explanation,Yes,NaN,5523897,"[-0.02903948351740837, -0.004012380726635456, ..."
3,2019-10-04,"Money transfer, virtual currency, or money ser...",Virtual currency,Fraud or scam,NaN,On Friday XX/XX/XXXX I searched in the web XXX...,NaN,JPMORGAN CHASE & CO.,TX,75043,NaN,Consent provided,Web,2019-10-04,Closed with explanation,Yes,NaN,3395771,"[-0.01851532608270645, 0.00250532990321517, 0...."
4,2022-01-09,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,"Dear Sir or Madam, I checked my personal credi...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NY,10305,NaN,Consent provided,Web,2022-01-09,Closed with explanation,Yes,NaN,5087511,"[-0.028908617794513702, -0.008078554645180702,..."
5,2022-09-17,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,This is proof Equifax is not doing a proper in...,NaN,"EQUIFAX, INC.",IL,XXXXX,NaN,Consent provided,Web,2022-09-17,Closed with explanation,Yes,NaN,5992227,"[-0.023111913353204727, -0.010436994023621082,..."
6,2022-12-09,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,This is not a duplicate nor is this complaint ...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,90277,NaN,Consent provided,Web,2022-12-09,Closed with explanation,Yes,NaN,6296745,"[-0.023795152083039284, -0.005494810175150633,..."
7,2016-09-22,Mortgage,Other mortgage,"Application, originator, mortgage broker",NaN,I contacted Wells fargo because they have a pr...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,MD,21060,NaN,Consent provided,Web,2016-09-22,Closed with explanation,Yes,No,2127486,"[-0.019727632403373718, -0.010635362938046455,..."
8,2015-11-25,Credit card,NaN,Application processing delay,NaN,Sometime in XXXX of 2015 I was approved for a ...,NaN,CAPITAL ONE FINANCIAL CORPORATION,FL,32773,NaN,Consent provided,Web,2015-11-25,Closed with explanation,Yes,No,1672639,"[-0.01234468910843134, -0.004265441559255123, ..."
9,2023-01-14,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,I acquired my credit report and asked the cred...,NaN,"EQUIFAX, INC.",NY,11236,NaN,Consent provided,Web,2023-01-14,Closed with explanation,Yes,NaN

In [56]:
#Safe df_train and df_test to azure blob storage
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
import uuid

# Define the connection string and container name
conn_str = "DefaultEndpointsProtocol=https;AccountName=entchatgptstr;AccountKey=gZss7VuVlbiwShlfML34GaeAw3QYEyicbZLF9D47zkGa/GCgx6jNGcNGBz0qqRa4Ci3MPqXrWI2O+AStm9gdMg==;EndpointSuffix=core.windows.net"
container_name = "pubmeddata"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get a reference to the blob container
container_client = blob_service_client.get_container_client(container_name)

# Get a reference to the blob
#blob_client = container_client.get_blob_client("df_train_embeddings10k.csv")

# Upload the blob
#blob_client.upload_blob(df_train.to_csv(index=False), overwrite=True)

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_test_embeddings10k.csv")

# Upload the blob
blob_client.upload_blob(df_test.to_csv(index=False), overwrite=True)





{'etag': '"0x8DB819158ACE75A"',
 'last_modified': datetime.datetime(2023, 7, 10, 22, 2, 26, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'\x1f^4p\x92jCh'),
 'client_request_id': '73df3348-1f6d-11ee-8840-07237768152c',
 'request_id': 'dd742f79-a01e-0141-4b7a-b3ea92000000',
 'version': '2021-08-06',
 'version_id': None,
 'date': datetime.datetime(2023, 7, 10, 22, 2, 25, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

In [91]:
#Safe df_train and df_test to azure blob storage
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
import uuid

# Define the connection string and container name
conn_str = "DefaultEndpointsProtocol=https;AccountName=entchatgptstr;AccountKey=gZss7VuVlbiwShlfML34GaeAw3QYEyicbZLF9D47zkGa/GCgx6jNGcNGBz0qqRa4Ci3MPqXrWI2O+AStm9gdMg==;EndpointSuffix=core.windows.net"
container_name = "pubmeddata"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get a reference to the blob container
container_client = blob_service_client.get_container_client(container_name)

# Get a reference to the blob
#blob_client = container_client.get_blob_client("df_train_embeddings10k.csv")

# Upload the blob
#blob_client.upload_blob(df_train.to_csv(index=False), overwrite=True)

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_test_embeddings10k.csv")

#download the blob
blob_string = blob_client.download_blob().content_as_text(encoding="utf-8")

# Load the string into a Pandas dataframe
df_test = pd.read_csv(io.StringIO(blob_string), encoding="utf-8")

In [2]:
#Safe df_train and df_test to azure blob storage
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
import uuid
import pandas as pd

# Define the connection string and container name
conn_str = "DefaultEndpointsProtocol=https;AccountName=entchatgptstr;AccountKey=gZss7VuVlbiwShlfML34GaeAw3QYEyicbZLF9D47zkGa/GCgx6jNGcNGBz0qqRa4Ci3MPqXrWI2O+AStm9gdMg==;EndpointSuffix=core.windows.net"
container_name = "pubmeddata"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get a reference to the blob container
container_client = blob_service_client.get_container_client(container_name)

# Get a reference to the blob
#blob_client = container_client.get_blob_client("df_train_embeddings10k.csv")

# Upload the blob
#blob_client.upload_blob(df_train.to_csv(index=False), overwrite=True)

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_train_embeddings10k.csv")

#download the blob
blob_string = blob_client.download_blob().content_as_text(encoding="utf-8")

# Load the string into a Pandas dataframe
df_train = pd.read_csv(io.StringIO(blob_string), encoding="utf-8")

NameError: name 'io' is not defined

In [ ]:
#count rows of df_test
print(len(df_train))

#check if df_test has the column Embeddings_complaint
df_train.head(10)

#check if Embeddings_complaint cintains empty lists
df_train[df_train['Embeddings_complaint'].apply(lambda x: len(x) == 0)]

In [96]:
#count rows of df_test
print(len(df_test))

#check if df_test has the column Embeddings_complaint
df_test.head(10)

#check if Embeddings_complaint cintains empty lists
df_test[df_test['Embeddings_complaint'].apply(lambda x: len(x) == 0)]

10000


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Embeddings_complaint
